In [ ]:
import pandas as pd
from pathlib import Path
import glob
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 200)

In [ ]:
def load_systems_dataframe(ulx_only=False, beamed=False, half_opening_l_45=False):
    systems_df_path = Path('../data/processed/all_systems_df.csv')
    df = pd.read_csv(systems_df_path)
    if ulx_only:
        df = df[df['Lx'] > 1E39]
    if beamed:
        df = df[df['b'] < 1]
    if half_opening_l_45:
        df = df[df['theta_half_deg'] < 45]
    df = df.reset_index()
    df = df.drop(columns=['index', 'Unnamed: 0'])
    return df

In [ ]:
systems_df = load_systems_dataframe(True, False, False)

In [ ]:
systems_df

In [ ]:
#I only ran these simulations to 0.4 BH_NS ratio before realising my method could be faster.
df_a = pd.read_csv('../data/processed/df_a_full.csv')

In [ ]:
df_a

In [ ]:
BH_NS_RATIO = 0.1
plt.xlabel('alive/dead ratio')
plt.ylabel('Number of systems')
plt.title('Transient ULX distribution')

for BH_NS_RATIO in df_a['BH_NS'].unique():
    df = df_a[(df_a['BH_NS'] == BH_NS_RATIO) & (df_a['ratio']!=0) & (df_a['ratio']!=1)]
    # df = df_a[(df_a['BH_NS'] == BH_NS_RATIO)]
    number_of_systems = len(df)
    plt.hist(df['ratio'], bins=50, label=round(BH_NS_RATIO, 2), density=True)
plt.legend()

In [ ]:
df

In [ ]:
my_list = list(systems_df.is_bh)
Z_list = list(systems_df.Z)

is_bh = [my_list[s] for s in df['system_num']]
Z = [Z_list[s] for s in df['system_num']]
df['is_bh'] = is_bh
df['Z'] = Z

In [ ]:
#I've realised that some of the systems have far less simulations on them than others, this needs to be sorted
print(df['system_num'].value_counts().sort_index())
print('==========')
print(df['system_num'].value_counts().sort_values())

In [ ]:
df['is_bh'].value_counts()

In [ ]:
#Example of poorly sampled system
df[df['system_num'] == 529].hist('inclination')

In [ ]:
inclinations = np.arange(91)
dincls = np.arange(1,46)
number_of_ulxs = 151

repeats = 10

number_of_required_simulations = len(inclinations)*len(dincls)*repeats*number_of_ulxs

time_per_simulation = 0.3
simulation_hours = number_of_required_simulations*time_per_simulation/3600

In [ ]:
simulation_hours

In [ ]:
cols = ['system_num', 'Z', 'is_bh', 'dincl', 'inclination', 'alive', 'dead', 'ratio']
df = df[cols]

In [ ]:
df

In [ ]:
df_sub = df[df['Z'] == 0.0002]
plt.hist2d(df_sub['dincl'], df_sub['ratio'], bins=45)

In [ ]:
systems_df['P_wind_years'] = systems_df['P_wind']/60/60/24/365

# Checking new lightcurve simulations

In [ ]:
csv_files = glob.glob('../src/new_curve_results/*.csv')
res = {}
for file in csv_files:
    res[file] = pd.read_csv(file)
df_new = pd.concat(res.values())
df_new = df_new.drop(['Unnamed: 0'], axis=1)
df_new['ratio'] = np.divide(df['alive'], (df['alive']+df['dead']))
df_new.loc[np.isnan(df_new['ratio']),'ratio']=0
df_new_transient = df_new[(df_new['ratio'] != 0) & (df_new['ratio'] != 1)]
df_new

In [ ]:
df_new_transient.drop_duplicates()

In [ ]:
df_unique = df_new[df_new.duplicated(subset=None, keep='first')]
df_new_transient = df_unique[(df_unique['ratio'] != 0) & (df_unique['ratio'] != 1)]
df_new['ratio'].value_counts()

In [ ]:
plt.hist2d(df_new_transient['dincl'], df_new_transient['ratio'], bins=45)

In [ ]:
df736 = df_new[df_new['system_id'] == 736]
df736 = df_new[df_new['inclination'] == 0]
df736 = df736.sort_values(by=['theta', 'inclination', 'dincl'])
df736 = df736[df736['theta'] == df736['theta'].unique()[0]]
df736

In [ ]:
df_new['system_id'].value_counts().sort_values()

In [ ]:
df_new_transient['ratio'].hist()

In [ ]:
for sysid in df_new['system_id'].unique():
    subset = df_new[df_new['system_id'] == sysid]
    subset['inclination'].hist(bins=91)

In [ ]:
np.sort(df_new['inclination'].unique())

In [ ]:
np.sort(df_new['theta'].unique())

In [ ]:
df_new['system_id'].value_counts().sort_values()

In [ ]:
df_new_transient['ratio'].hist()

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
plt.hist2d(df_new['dincl'], df_new['ratio'], bins=45)

In [ ]:
subset = df_new_transient[df_new_transient['system_id']==883].sort_values(by=['inclination', 'dincl'])
plt.hist2d(subset['dincl'], subset['ratio'], bins=45)

# How many ULXs will eROSITA see?

A first calculation we may perform is what percentage of our ULX population we will be able to see purely due to beaming.

the beaming factor, b, for random inclination essentially provides the probability of observing the source down the cone, since it is a ratio of the solid angle of a sphere to the size subtended by the two cones.

We may hence obtain a very crude upper limit for the number of ULXs observed by eRosita in the following manner:

For our sample of binary systems, we can calculate what percentage of them are ULXs, and what percentage of them are above the eROSITA detection threshold.

If we know how many binary systems eROSITA is predicted to observe, we can find what percentage of them would be ULXs from greg's population synthesis, 

number_of_ulxs/number_of_systems_above_erosita_threshold = number_of_observed_ulxs/number_of_observed_binary_systems

https://www.eso.org/sci/meetings/2012/surveys2012/Presentations/Day4-Thursday/Merloni.pdf slide 18 has the eROSITA limits as a function of each observing cycle in units erg/s/cm^2

https://www.aanda.org/articles/aa/pdf/2014/07/aa23766-14.pdf performed simulations on the number of XRBs that would be detected by eROSITA



In [ ]:
#ULXs only
all_systems_df = load_systems_dataframe(False, False, False)
systems_df = load_systems_dataframe(True, False, False)
print(f'number of binary systems: {len(all_systems_df)}')
print(f'number of ULXs: {len(systems_df)} ({round(len(systems_df)/len(all_systems_df)*100, 3)}% of all systems)')
b_sum = systems_df['b'].sum()
print(f'number of visible ULXs: {round(b_sum,0)} ({round(b_sum/len(systems_df)*100,2)}% of all ULXs)')

In [ ]:
#EROSITA_stats
erosita = pd.DataFrame()
erosita['cycle_number'] = [1,2,3,4,8]
erosita['f_lim'] = [4.5E-14, 2.8E-14, 2.1E-14, 1.8E-14, 1.1E-14] #0.5 - 2Kev erg/cm^2/s
erosita

In [ ]:
systems_df.iloc[988]

# Earnshaw ULX

In [ ]:
from astropy.io import fits

In [ ]:
ulx_file = '../data/external/Earnshaw_ULX_cat/earnshaw_Xraycatalogue.fits'
with fits.open(ulx_file) as hdul:
    #hdul.info()
    data = pd.DataFrame(hdul[1].data)

In [ ]:
data

In [ ]:
for i in data.columns:
    print(i)

In [ ]:
#Sources with more than 1 obervation
sources = data['SRCID'].value_counts()[data['SRCID'].value_counts() > 1].sort_values(ascending=False)

In [ ]:
src901 = data[data['SRCID'] == 901].sort_values(by=['MJD_START'])
src901

In [ ]:
def classify_ULX(subset):
    if max(subset['EP_8_LUMINOSITY']) > 1E39 and min(subset['EP_8_LUMINOSITY']) < 1E39:
        
        return 

def is_always_off(subset):
    return max(subset['EP_8_LUMINOSITY']) < 1E39

def is_always_on(subset):
    return 



In [ ]:
number_of_transients = 0
number_of_dead = 0
number_of_alive = 0

for source in sources.index:
    subset = data[data['SRCID'] == source].sort_values(by=['MJD_START'])
    if max(subset['EP_8_LUMINOSITY']) > 1E39 and min(subset['EP_8_LUMINOSITY']) < 1E39: #TRANSIENT
        plt.figure(figsize=(15,3))
        plt.ylabel('Lx')
        plt.xlabel('MJD')
        plt.title(subset['IAUNAME'].unique()[0] + ' TRANSIENT')
        plt.errorbar(subset['MJD_START'], subset['EP_8_LUMINOSITY'], yerr=subset['EP_8_LUMINOSITY_ERR'], fmt='none', capsize=1.5)
        plt.axhline(1E39, c='red')
        number_of_transients+=1
    elif max(subset['EP_8_LUMINOSITY']) < 1E39: #DEAD
        number_of_dead+=1
    elif min(subset['EP_8_LUMINOSITY']) > 1E39: #Alive
        number_of_alive+=1

In [ ]:
print(f'Objects in Earnshaw cat: {len(data)}')
print(f'Objects with more than 1 observation: {len(sources)}')
print(f'Number of Transient ULXs: {number_of_transients}')
print(f'Number of alive ULXs: {number_of_alive}')
print(f'Number of dead ULXs: {number_of_dead}')

In [ ]:
plt.pie([number_of_alive, number_of_dead, number_of_transients], labels=['alive', 'dead', 'transient'], autopct='%1.1f%%')

In [ ]:
plt.pie([number_of_alive, number_of_transients], labels=['alive', 'transient'], autopct='%1.1f%%')